In [4]:
import websocket,json
import _thread
import time
import rel
from datetime import datetime
from datetime import timedelta
import math

from IPython.display import clear_output
from binance.client import Client

APIKey = "QbuTxekA4Akzq0OPkGifpztmPePHKRYkCNNDsvu0GKvVJK3QCHAomiMzGpGxGh38";
SecretKey = "8wd3erOoDXWYTKXNye9G0aB22olJJb7iXMsls0PFfcFaXTUkogcfwxgT2EbumhDL";
symbol1 = 'btcusdt'
symbol2 = 'ethusdt'
symbol3 = 'ethbtc'
round1 = 2 ;
round2 = 2 ;
round3 = 5 ;
RoundQuantity = 4
RoundQuantity2 = 5
RoundQuantity3 = 4
SOCK = 'wss://stream.binance.com:9443/stream?streams='+symbol1+'@bookTicker/'+symbol2+'@bookTicker/'+symbol3+'@bookTicker'
client = Client(APIKey,SecretKey)
def current_milli_time():
    return round(time.time() * 1000)

In [5]:
client.get_account()
t0 = current_milli_time()
symbol = 'BTCUSDT'
ticker = client.get_symbol_ticker(symbol=symbol)
ticker_price = float(ticker['price'])
t1 = current_milli_time()
ticker,ticker_price,str(t1-t0)+'ms'

def BuyOrder(sym,quantity): # BTCUSDT quantity of BTC
    t0 = current_milli_time()
    order = client.order_market_buy(symbol=sym,quantity = quantity)
    tm = current_milli_time() - t0
    print(f'Buy Order Done in {tm}ms :{order}')
    return order
def SellOrder(sym,quantity):  # BTCUSDT quantity of BTC
    t0 = current_milli_time()
    order = client.order_market_sell(symbol=sym,quantity = quantity)
    tm = current_milli_time() - t0
    print(f'Sell Order Done in {tm}ms :{order}')
    return order

In [7]:
def GetBalance(get_account,crypto):
    Balance = client.get_account()['balances']
    res = '0'
    for x in Balance:
        if(x['asset']==crypto):
            res = x['free']
    return float(res)
def ceil(Balance,Round):
    res = Balance
    if('e' not in str(Balance) ):
        res = str(Balance)+'xlljj'
        res = res[:res.index('.')+Round+1]
    return float(res)

In [8]:
Balance = GetBalance(client.get_account(),'ETH') 
BalanceRound = ceil(Balance,RoundQuantity)
Balance,BalanceRound

(0.04096579, 0.0409)

In [74]:
print(f'Start Getting pricess')
iTime = current_milli_time()
Balance = GetBalance(client.get_account(),'ETH')  
ticker2 = client.get_symbol_ticker(symbol='BTCDAI')  
ticker3 = client.get_symbol_ticker(symbol='ETHBTC')
ms = current_milli_time() - iTime
print(f'Done Get Prices ... in {ms}ms')
print(f'BTCDAI Price {ticker2}')
print(f'ETHBTC Price {ticker3}')

BalanceRound = ceil(Balance,RoundQuantity)
print(f'Starting Balance:{Balance}ETH -> {BalanceRound}ETH')
QuoteQty,QuoteQtyRound,commission=0,0,0
def HFT3Leg(StartBalance,Leg1,Leg2,Leg3):
    global QuoteQtyRound,QuoteQty,BalanceRound
    try:
        order = SellOrder(Leg1,StartBalance) 
        cummulativeQuoteQty = float(order['cummulativeQuoteQty'])
        commission = order['fills']
        commission = float(commission[0]['commission'])
        QuoteQty = cummulativeQuoteQty - commission
        QuoteQtyRound = ceil(QuoteQty,RoundQuantity+2) 
        print('QuoteQtyRound: '+str(QuoteQtyRound))
    except Exception as e:
        print(f"An error occurred: {e}")
        info = client.get_symbol_info(Leg1)
        

    print('--')
    print('--') 
    ticker_price2 = float(ticker2['price'])
    Balance = QuoteQtyRound/ticker_price2
    BalanceRound = ceil(Balance,RoundQuantity2) 
    try:
        order = BuyOrder(Leg2,BalanceRound) 
        cummulativeQuoteQty = float(order['cummulativeQuoteQty'])
        commission = order['fills']
        commission = float(commission[0]['commission'])
        QuoteQty = cummulativeQuoteQty - commission
        QuoteQtyRound = ceil(QuoteQty,RoundQuantity+2)
    except Exception as e:
        print(f"An error occurred: {e}")
        info = client.get_symbol_info(Leg1)
    
    BalanceRound=BalanceRound-(BalanceRound*0.001)
    print('QuoteQtyRound: '+str(QuoteQtyRound))
    print('BalanceRound: '+str(BalanceRound)) 

    
    print('--')
    print('--') 
    ticker_price3 = float(ticker3['price'])
    Balance = BalanceRound/ticker_price3
    BalanceRoundX = ceil(Balance,RoundQuantity3) 
    try:
        order = BuyOrder(Leg3,BalanceRoundX) 
        cummulativeQuoteQty = float(order['cummulativeQuoteQty'])
        commission = order['fills']
        commission = float(commission[0]['commission'])
        QuoteQty = cummulativeQuoteQty - commission
        Balance = Balance - commission
    except Exception as e:
        print(f"An error occurred: {e}")
        info = client.get_symbol_info(Leg1)
    
     
    print('QuoteQtyRound: '+str(QuoteQtyRound))
    print('BalanceRoundX: '+str(BalanceRoundX))
    print('Balance: '+str(Balance))

print(f'Start  3Leg ...')
iTime = current_milli_time() 
HFT3Leg(BalanceRound,'ETHDAI','BTCDAI','ETHBTC')
ms = current_milli_time() - iTime
print(f'Done 3Leg ... in {ms}ms')

Start Getting pricess
Done Get Prices ... in 1276ms
BTCDAI Price {'symbol': 'BTCDAI', 'price': '68352.04000000'}
ETHBTC Price {'symbol': 'ETHBTC', 'price': '0.03591000'}
Starting Balance:0.04547595ETH -> 0.0454ETH
Start  3Leg ...
Sell Order Done in 297ms :{'symbol': 'ETHDAI', 'orderId': 840098286, 'orderListId': -1, 'clientOrderId': 'x-HNA2TXFJa561b07b6444fef68dff6a', 'transactTime': 1730629477883, 'price': '0.00000000', 'origQty': '0.04540000', 'executedQty': '0.04540000', 'cummulativeQuoteQty': '111.50557800', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'workingTime': 1730629477883, 'fills': [{'price': '2456.07000000', 'qty': '0.04540000', 'commission': '0.11150558', 'commissionAsset': 'DAI', 'tradeId': 13984491}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}
QuoteQtyRound: 111.394072
--
--
Buy Order Done in 307ms :{'symbol': 'BTCDAI', 'orderId': 548478304, 'orderListId': -1, 'clientOrderId': 'x-HNA2TXFJ7c38e2dcf5e363dd31d513', 'transactTime': 1730629478

In [43]:
BalanceRound = 0.0001998
BalanceRound

0.0001998

In [49]:
ticker = client.get_symbol_ticker(symbol='ETHBTC')
ticker_price = float(ticker['price'])
Balance = BalanceRound/ticker_price
BalanceRoundX = ceil(Balance,RoundQuantity3) 
BalanceRound,ticker_price,Balance,BalanceRoundX

(0.0001998, 0.03589, 0.005567010309278351, 0.0055)

In [47]:

try:
    order = BuyOrder('ETHBTC',BalanceRoundX) 
    cummulativeQuoteQty = float(order['cummulativeQuoteQty'])
    commission = order['fills']
    commission = float(commission[0]['commission'])
except Exception as e:
    print(f"An error occurred: {e}")
    info = client.get_symbol_info('ETHBTC')

print('QuoteQtyRound: '+str(QuoteQtyRound))
print('BalanceRound: '+str(BalanceRound))

An error occurred: APIError(code=-1100): Illegal characters found in parameter 'quantity'; legal range is '^([0-9]{1,20})(\.[0-9]{1,20})?$'.
QuoteQtyRound: 13.659447
BalanceRound: 0.0001998


In [ ]:
ethbtc -> 0.03591 , 0.0359 (1e-05)
ethdai -> 2462.78 , 2459.92 (2.86)
btcdai -> 68572.51 , 68449.87 (122.64)
Max ->  0.03591 , 2462.78 , 68572.51
Min -> 0.03591 , 2462.24 , 68573.66
Max:1.00014 Min:0.9999
1.0001385479693041
Starting Balance:0.04355989ETH -> 0.0435ETH
Start  3Leg ...
Sell Order Done in 12ms :{'symbol': 'ETHDAI', 'orderId': 840114676, 'orderListId': -1, 'clientOrderId': 'x-HNA2TXFJb9f24a3ac70a2412665fbc', 'transactTime': 1730632662054, 'price': '0.00000000', 'origQty': '0.04350000', 'executedQty': '0.04350000', 'cummulativeQuoteQty': '107.00652000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'workingTime': 1730632662054, 'fills': [{'price': '2459.92000000', 'qty': '0.04350000', 'commission': '0.10700652', 'commissionAsset': 'DAI', 'tradeId': 13984545}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}
QuoteQtyRound: 106.899513
--
--
Buy Order Done in 9ms :{'symbol': 'BTCDAI', 'orderId': 548490207, 'orderListId': -1, 'clientOrderId': 'x-HNA2TXFJb87562f61463dc588a0467', 'transactTime': 1730632662063, 'price': '0.00000000', 'origQty': '0.00155000', 'executedQty': '0.00155000', 'cummulativeQuoteQty': '106.29058550', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': 1730632662063, 'fills': [{'price': '68572.51000000', 'qty': '0.00017000', 'commission': '0.00000017', 'commissionAsset': 'BTC', 'tradeId': 6519516}, {'price': '68573.66000000', 'qty': '0.00108000', 'commission': '0.00000108', 'commissionAsset': 'BTC', 'tradeId': 6519517}, {'price': '68579.02000000', 'qty': '0.00030000', 'commission': '0.00000030', 'commissionAsset': 'BTC', 'tradeId': 6519518}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}
QuoteQtyRound: 106.290585
BalanceRound: 0.00154845
--
--
Buy Order Done in 8ms :{'symbol': 'ETHBTC', 'orderId': 4156487962, 'orderListId': -1, 'clientOrderId': 'x-HNA2TXFJ1b3c01a483fc8c677b58c6', 'transactTime': 1730632662072, 'price': '0.00000000', 'origQty': '0.04310000', 'executedQty': '0.04310000', 'cummulativeQuoteQty': '0.00154772', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': 1730632662072, 'fills': [{'price': '0.03591000', 'qty': '0.04310000', 'commission': '0.00004310', 'commissionAsset': 'ETH', 'tradeId': 471567288}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}
QuoteQtyRound: 106.290585
BalanceRoundX: 0.0431
Balance: 0.04307720075187971
Done 3Leg ... in 29ms